<h3>Importing the required libraries</h3>
<p>Loading the libraries that are going to be used during the analysis.</p>

In [1]:
#Importing the required libraries.
import math
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client as client
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import sys
import scipy.stats
import csv
import inspect
%run -i '/home/juan-david/Documents/data_science/course_openclassrooms/projet_3/livrables/p3_nettoyage_fonctions_aide.py'

<h3>1. Loading the database's dataframe </h3>
<p>The open food facts file is stored as a dask object due to its size.</p>

In [2]:
filename = '/home/juan-david/Documents/data_science/course_openclassrooms/projet_3/donnees/en.openfoodfacts.org.products.tsv'
data = dd.read_csv(filename, delimiter = "\t", dtype={'allergens': 'object',
                                                      'code' : 'object',
                                                      'allergens_en':'object',
                                                      'cities':'object',
                                                      'created_t':'object',
                                                      'categories':'object',
                                                      'last_modified_t':'object',
                                                      'categories_en': 'object',
                                                      'categories_tags': 'object',
                                                      'cities_tags': 'object',
                                                      'emb_codes': 'object',
                                                      'emb_codes_tags': 'object',
                                                      'first_packaging_code_geo': 'object',
                                                      'generic_name': 'object',
                                                      'image_small_url': 'object',
                                                      'image_url': 'object',
                                                      'ingredients_from_palm_oil_tags': 'object',
                                                      'ingredients_that_may_be_from_palm_oil_tags': 'object',
                                                      'labels': 'object',
                                                      'labels_en': 'object',
                                                      'labels_tags': 'object',
                                                      'main_category': 'object',
                                                      'main_category_en': 'object',
                                                      'manufacturing_places': 'object',
                                                      'manufacturing_places_tags': 'object',
                                                      'origins': 'object',
                                                      'origins_tags': 'object',
                                                      'packaging': 'object',
                                                      'packaging_tags': 'object',
                                                      'pnns_groups_1': 'object',
                                                      'pnns_groups_2': 'object',
                                                      'purchase_places': 'object',
                                                      'stores': 'object',
                                                      'traces': 'object',
                                                      'traces_en': 'object',
                                                      'traces_tags': 'object'})

<h3>2.1 Analysis of the open food facts dataframe "en.openfoodfacts.org.products.tsv"</h3>
<p>The dataframe contains information about products, their ingredients, additives, and nutrition scores.</p>
<p>Analysing the principal characteristics of the dataframe.</p>
<ol>
<p>-Structure remarks.</p> 
<p>-Shape.</p>
<p>-Data types.</p>  
<p>-Identifying the amount null values, if possible.</p>   
<p>-Others.</p>
</ol>

In [3]:
#Dataframe visual structure.
data.head(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,0000000004559,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,0000000016087,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,0000000016094,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h3>Structure remarks</h3>
<ol>
<p>- Within the dataframe, fields finishing by _t, and _datetime correspond to dates.</p>
<p>- Within the dataframe, fields finishing by _tags are comma separated list of tags.</p> 
<p>- Within the dataframe, fields finishing by _100g correspond to the amounts of a nutrient for 100g of product.</p> 
<p>- Within the dataframe, fields finishing by _serving correspond to the amount of a nutrient for 1 product unit.</p> 
<p>- Nutrition-score-fr_100g is the nutrition score adapted for the french market.</p> 
<p>- Nutrition-score-UK_100g is the nutrition score defined by the UK Food Standards Administration.</p> 
</ol>

In [4]:
#Dataframe shape.
print((len(data),data.shape[1]))

(356027, 163)


In [5]:
#Data types in dataframe.
print(data.dtypes)

code                        object
url                         object
creator                     object
created_t                   object
created_datetime            object
                            ...   
carbon-footprint_100g      float64
nutrition-score-fr_100g    float64
nutrition-score-uk_100g    float64
glycemic-index_100g        float64
water-hardness_100g        float64
Length: 163, dtype: object


In [ ]:
#Identifying the amount of null values in the dataframe.
print(cal_nan_percents(data).head(15))

<h3>2.2 Data cleaning of the open food facts file "en.openfoodfacts.org.products.tsv"</h3>
<p>Several processes are done during this phase : Irrelevant columns are deleted, obtaining ingredients from products, filtering only french entries, removing almost empty rows, removing almost empty columns, removing duplicated labels, etc.</p>
</ol>

In [6]:
#Rename all columns that contain a "-" since in some scenarios this can cause problems with python 3.
data = data.rename(columns={'nutrition-score-fr_100g': 'nutrition_score_fr_100g',
                            'nutrition-score-uk_100g': 'nutrition_score_uk_100g',
                            'vitamin-c_100g': 'vitamin_c_100g',
                            'vitamin-a_100g': 'vitamin_a_100g',
                            'saturated-fat_100g': 'saturated_fat_100g',
                            'trans-fat_100g': 'trans_fat_100g'})

<h3>Removing irrelevant columns</h3>
<p>- Within the dataframe, fields finishing by _t, and _datetime correspond to dates.</p>
<p>- The column salt_100g and sodium_100g are the same. One of them is leaved, the other is removed.</p>

In [7]:
#Dropping all columns which are not interesting with the intend of the project.
#salt_100g
#Countries c'est important.
data = data.drop(columns=['creator',
                          'brands',
                          'brands_tags',
                          'created_datetime',
                          'last_modified_t',
                          'last_modified_datetime',
                          #'categories',
                          'main_category',
                          'additives',
                          'additives_tags',
                          #'categories_tags',
                          'states',
                          'states_en',
                          'states_tags',
                          'url',
                          'quantity',
                          'packaging_tags',
                          'packaging',
                          'created_t',
                          'last_modified_t',
                          'ingredients_from_palm_oil_n', 
                          'ingredients_that_may_be_from_palm_oil_n',
                          'pnns_groups_1',
                          'pnns_groups_2',
                          'image_url',
                          'image_small_url',
                          'salt_100g',
                         ])
#New dataframe shape.
#print((len(data),data.shape[1]))

In [8]:
#Dataframe shape.
print((len(data),data.shape[1]))

(356027, 139)


<h3>Filtering data to obtain only french entries </h3>
<p>Selecting from the dataframe only french products.</p>

In [9]:
#Checking if the product is comming from France.
contain_france = is_product_sold_in_france(data)

#Create a mapping to filter french products.
data = create_mapping_is_product_in_france(data,contain_france)

#Filter dataframe to obtain only french entries.
data = data[(data['in_france'] == True)]

#Reconvert empty strings in "ingredients_text" to np.nan values.
data['ingredients_text'] = data['ingredients_text'].replace(r'^\s*$', np.NaN, regex=True)

In [10]:
#Dataframe shape.
print((len(data),data.shape[1]))

(129320, 140)


<h3>Guaranteeing enough amount of data in columns</h3>
<p>- Assuring rows contain minimal information like product_name, ingredients, category, and nutrition scores.</p>
<p>- Assuring that only rows having a certain number of non-empty columns remain in the dataframe.</p>
<p>- Assuring that columns within the dataframe aren't empty, and/or contain a minimal number of row values.</p>

<h3>Removing rows that don't allow to identify products and their charaterisics</h3>
<p>The row should be removed when there is not information for columns such as : product_name, ingredients_text, categories, and nutrition_score_fr_100g.</p>

In [11]:
#If a label does not contain a the product's name, then it should be removed.
data = data.query('product_name == product_name')

#If a label does not contain a the ingredients_text value,then it should be removed.
data = data.query('ingredients_text == ingredients_text')

#If a label does not contain a categroy, it should be removed.
data = data.query('categories == categories')

#If a label does not contain a nutrition score calification, it should be removed.
data = data.query('nutrition_score_fr_100g == nutrition_score_fr_100g')

#print((len(data),data.shape[1]))
data.head(3)

,code,product_name,generic_name,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,...,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition_score_fr_100g,nutrition_score_uk_100g,glycemic-index_100g,water-hardness_100g,in_france
185,0000007020254,Root Beer,NaN,"Boissons,Boissons gazeuses,Sodas,Boissons sucr...","en:beverages,en:carbonated-drinks,en:sodas,en:...","Beverages,Carbonated drinks,Sodas,Sugared beve...",NaN,NaN,États-Unis,etats-unis,...,NaN,NaN,NaN,NaN,NaN,18.0,3.0,NaN,NaN,True
190,0000010068175,Thé noir aromatisé violette et fleurs,NaN,en:beverages,"en:beverages,en:non-sugared-beverages","Beverages,Non-sugared beverages",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,True
231,0000040608754,"Pepsi, Nouveau goût !",Boisson gazeuse rafraîchissante aux extraits n...,Sodas au cola,"en:beverages,en:carbonated-drinks,en:sodas,en:...","Beverages,Carbonated drinks,Sodas,Colas,Sugare...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.0,2.0,NaN,NaN,True


In [12]:
#Dataframe shape.
print((len(data),data.shape[1]))

(53242, 140)


<h3>Assuring minimal amount of data in rows</h3>
<p>The row should be removed when there is not information for columns such as : product_name, ingredients_text, categories, and nutrition_score_fr_100g.</p>

In [13]:
#Deleting rows having insuffient information in the dataframe.
#Rows may have at least an "n_100g_cols" of non_empty _100g columns within the dataframe. 
n_100g_cols = 12
data = delete_rows_empty_100g_columns(data, n_100g_cols)

#The column that was used to remove rows with insuffcient information is dropped.
data = data.drop(columns=['num_100g_non_empty_columns'])
print((len(data),data.shape[1]))

/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/dask/dataframe/multi.py:1213: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"


(3138, 140)


<h3>Assuring minimal amount of data in columns</h3>
<p>Columns having the majority of entries as null, are removed.</p>
<p>The removal process is taken into account by a boundary percentage limit of null values within a column.</p>

In [14]:
#Deleting columns having insuffient information in the dataframe..
na_percents = cal_nan_percents(data)
list_almost_empty_columns, list_columns_100_g_nan  = return_almost_and_null_nan_columns(na_percents, obtain_g_columns(data), 0.20)
data = remove_almost_and_null_columns(data,list_almost_empty_columns, list_columns_100_g_nan)

In [15]:
print((len(data),data.shape[1]))

(3138, 89)


<h3>Removing duplicate entries in the dataframe (if there are any)</h3>

In [12]:
data.drop_duplicates()
#New dataframe shape.
#print((len(data),data.shape[1]))

,code,product_name,categories,categories_tags,categories_en,countries,countries_tags,countries_en,ingredients_text,additives_n,nutrition_grade_fr,main_category_en,energy_100g,energy-from-fat_100g,fat_100g,saturated_fat_100g,-arachidic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,trans_fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,sodium_100g,alcohol_100g,vitamin_a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin_c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,nutrition_score_fr_100g,nutrition_score_uk_100g,in_france,num_100g_non_empty_columns
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
print((len(data),data.shape[1]))

(3138, 90)


<h3>3. Obtaining from the 'ingredients_text' column the proportion of healthy composals </h3>
<p>Calculating, from the ingredients of a product, the proportion of healthy composals in benefit of the cardiovascular health.</p>
<p> - A list with heart-healthy-foods is provided to check wether the product ingredients are healthy or not<p/>
<p> - This proportion is stored in a new column in the dataframe<p/>

<h3>Defining ingredients that are beneficial for the heart health</h3>
<p>A list of healthy ingredients should be provided.</p>

In [14]:
#Create binary columns for healthy ingredients.
heart_healthy_foods = ['épinard','spinach','grape','raisin','water,''eau','tomate','légume','chou', 'kale', 'collard','blé', 'wheat', 'oats','avoine', 'barley','orge', 'quinoa', 'berries', 'avocado','avocat','saumon', 'salmon', 'tuna','thon', 'walnuts','noix', 'nuts','cherries','cerise','haricot', 'beans', 'chocolate','chocolat', 'tomato', 'almonds','amande','lettuce','salade','broccoli','brocoli','cucumber','cocombre','seeds','graines', 'garlic','ail', 'olive oil','olive','edamame','tofu','sweet potatoe','patate douce','orange','thé', 'green tea','apple','pomme','']
#Replace Nan values in the database ingredients column.
data['ingredients_text'] = data['ingredients_text'].fillna("")

In [15]:
#Create binary of healthy ingredient columns
data = create_binary_columns_for_heart_health_products(data, heart_healthy_foods)
print(data.shape)

(Delayed('int-1f6b844b-b6ef-4bed-ad42-af07a2b89244'), 148)


<h3>Obtaining the sum of healthy products</h3>
<p>Calculating how many ingredients from the ingredients of a product are beneficial for the health.</p>

In [16]:
#Creating the column that counts the number of healthy ingredients in the product.
data = add_sum_heart_healthy_foods_per_product_to_df(data,heart_healthy_foods)

#Deleting the binary columns of ingredients from the dataframe.
data = data.drop(columns=obtain_binary_columns_heart_healthy_foods_in_df(data))
print(data.shape)

(Delayed('int-4682d3fa-a9eb-43b9-aacc-88fe01e52709'), 149)
(Delayed('int-cb00ff50-b4ad-4285-aade-61115786e2ff'), 91)


/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/dask/dataframe/multi.py:1213: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"


<h3>Calculating the proportion of healthy ingredients in each product</h3>
<p>For a product, the number of healthy ingredients out of the total number of ingredients is calculated.</p>
<p>The information is stored as a new column named "proportion_health_composals".</p>

In [17]:
#Creating proportion of healthy products column in the dataframe.
data = obtain_proportion_healthy_composals(data)
print(data.shape)

(Delayed('int-7c09570c-be72-4f22-a69f-b648601b3909'), 92)


In [28]:
#Check the number of null values in columns at the end of the exercise.
pd.set_option("display.max_rows", None, "display.max_columns", None)
na_percents = cal_nan_percents(data)
na_percents

,columns_names,Nan_percent
0,code,0.000000
1,product_name,0.000000
2,categories,0.000000
3,categories_tags,0.000000
4,categories_en,0.000000
5,countries,0.000000
6,countries_tags,0.000000
7,countries_en,0.000000
8,ingredients_text,0.000000
9,additives_n,0.000000


In [19]:
#Check the dataframe shapes at the end of the exercise.
print((len(data),data.shape[1]))

(3138, 92)


In [29]:
#Export the dataframe to a csv file to start the exploration phase in a separate notebook.
import os
path_d = '/home/juan-david/Documents/data_science/course_openclassrooms/projet_3/donnees'
data.to_csv(os.path.join(path_d, 'donnees_analyse_exploratoire.csv'))  

['/home/juan-david/Documents/data_science/course_openclassrooms/projet_3/donnees/donnees_analyse_exploratoire.csv/0.part',
 '/home/juan-david/Documents/data_science/course_openclassrooms/projet_3/donnees/donnees_analyse_exploratoire.csv/1.part',
 '/home/juan-david/Documents/data_science/course_openclassrooms/projet_3/donnees/donnees_analyse_exploratoire.csv/2.part']